### Importing Necessary Libraries

In [9]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#### Load the dataset

In [5]:
twitter=pd.read_csv(r"C:\Users\DELL\Downloads\archive (8)\training.1600000.processed.noemoticon.csv",encoding="latin-1",header=None)

In [12]:
#checking the number of rows and columns
twitter.shape

(1600000, 6)

First 5 rows of the dataset

In [6]:
twitter.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [11]:
# printing the stopwords in English
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

#### Data Preprocessing

In [13]:
# naming columns and reading the dataset again

column_names=["target","id","date","flag","user","text"]
twitter=pd.read_csv(r"C:\Users\DELL\Downloads\archive (8)\training.1600000.processed.noemoticon.csv",names=column_names,encoding="latin-1",)

In [14]:
twitter.shape

(1600000, 6)

In [15]:
# printing first 5 columns of te dataset again
twitter.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [16]:
# checking whether there are missing values in the dataset
twitter.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

There are no missing values

In [17]:
# checking the distribution of target column
twitter["target"].value_counts()

0    800000
4    800000
Name: target, dtype: int64

The dataset is balanced.The dataset contains both positive and negative tweets equally
0- Negative tweet
1-Positive tweet

In [18]:
# convert the target "4" to "1"
twitter.replace({"target":{4:1}},inplace=True)

In [19]:
# checking the distribution of target column again
twitter["target"].value_counts()

0    800000
1    800000
Name: target, dtype: int64

0 --> Negative Tweet

1 -->Positive Tweet

#### Stemming

Stemming is the process of reducing a word to its root word

In [20]:
port_stem=PorterStemmer()

In [21]:
def stemming(content):
    stemmed_content=re.sub("[^a-zA-Z]"," ",content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content=" ".join(stemmed_content)
    return stemmed_content

In [22]:
twitter["stemmed_content"]=twitter["text"].apply(stemming)

In [23]:
twitter.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [27]:
# Seperating the data and label
x=twitter["stemmed_content"]    # this is the data
y=twitter["target"]             # this is labels

Splitting the data to training data and test data

In [28]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=5,stratify=y)

In [29]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [30]:
# converting the textual data to numerical data

vectorizer=TfidfVectorizer()

x_train_f=vectorizer.fit_transform(x_train)
x_test_f=vectorizer.transform(x_test)

In [32]:
x_train_f

<1280000x461469 sparse matrix of type '<class 'numpy.float64'>'
	with 9453812 stored elements in Compressed Sparse Row format>

Training the Machine Learning Model

Logistic Regression

In [33]:
model=LogisticRegression(max_iter=1000)

In [34]:
model.fit(x_train_f,y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy Score

In [35]:
# accuracy score on training data
x_train_pred=model.predict(x_train_f)
training_data_accuracy=accuracy_score(y_train,x_train_pred)

In [36]:
print("Accuracy Score on the training data:",training_data_accuracy)

Accuracy Score on the training data: 0.8104046875


In [37]:
# accuracy score on test data
x_test_pred=model.predict(x_test_f)
test_data_accuracy=accuracy_score(y_test,x_test_pred)

In [38]:
print("Accuracy Score on the test data:",test_data_accuracy)

Accuracy Score on the test data: 0.77763125


Model Accuracy =77.8%

Saving the trained model

In [39]:
import pickle

In [40]:
filename="trained_model_for_twitter_sentimental_analysis.sav"
pickle.dump(model,open(filename,"wb"))

Using the saved model for future predictions

In [41]:
# loading the saved model
loaded_model=pickle.load(open(r"C:\Users\DELL\Prodigy\trained_model_for_twitter_sentimental_analysis.sav","rb"))